# Úkol č. 1 - vizualizace dat a web scraping (do 20. října)

  * V rámci tohoto úkolu musíte stáhnout dat z webu (tzv. _web scraping_, velmi základní) a následně data zpracovat a vizualizovat.
  * Cílem bude stáhnout data ze serveru https://www.volby.cz týkající se voleb do zastupitelstva Vámi vybraného (většího) města, uložit data o závěrečných pracích v tabulkovém formátu a pak vymyslet vizualizace a zobrazení dat, které umožní orientaci v těchto datech a zvýrazní zajímavé informace a zobrazit přehledně časový vývoj různých veličin.
 
> **Úkoly jsou zadány tak, aby Vám daly prostor pro invenci. Vymyslet _jak přesně_ budete úkol řešit, je důležitou součástí zadání a originalita či nápaditost bude také hodnocena!**

## Výběr zdroje dat

Vyberte si větší město, které má zastupitelstvo druhu 3 (Zastupitelstvo statutárního města) a strojově stáhněte informace o stranách a kandidátkách z následujících let:
 * [2002](https://www.volby.cz/pls/kv2002/kv12?xjazyk=CZ&xid=0), [2006](https://www.volby.cz/pls/kv2006/kv12?xjazyk=CZ&xid=0), [2010](https://www.volby.cz/pls/kv2010/kv12?xjazyk=CZ&xid=0), [2014](https://www.volby.cz/pls/kv2014/kv12?xjazyk=CZ&xid=0) a [2018](https://www.volby.cz/pls/kv2018/kv12?xjazyk=CZ&xid=0).
 

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **8 bodů**:
  * Strojově stáhněte data pro vybrané město a uložte je všechny do (asi dvou) přehledných tabulek ve formátu _csv_.
  * Data musí obsahovat _alespoň_ toto:
    * Vývoj výsledků (v procentech i počtu hlasů) pro jednotlivé strany v jednotlivých letech.
    * Seznam všech kandidátů všech stran v jednotlivých letech, u kandidáta by mělo být zaznamenáno: jméno, věk v době voleb, navrhující strana, politická příslušnost, volební zisk (procento i počet hlasů), pořadí na kandidátce, pořadí zvolení, jestli získal mandát (tyto informace získáte souhrnně ve _jmenných seznamech_).
  * V druhé části Vašeho Jupyter notebooku pracujte s těmito tabulkami načtenými z _csv_ souboru (aby opravující nemusel spouštět stahování z webu).
  * Tabulky ve formátu _csv_ také odevzdejte.
  * S využitím vybraných nástrojů zpracujte data a vymyslete vizualizace a grafy, aby bylo vidět následující:
    * Časový vývoj (po rocích voleb) počtu kandidujících stran i lidí a to celkově i po jednotlivých stranách (ve volbách, kterých se daná strana účastnila).
    * Věkovou strukturu kandidátů celkově i za jednotlivé strany a vývoj této struktury během jednotlivých voleb.
    * Časový vývoj volební účasti a volebních výsledků jednotlivých stran.
    * Časový vývoj podílu kandidujících s titulem a bez titulu.

**Další body zadání** za případné další body (můžete si vybrat, maximum bodů za úkol je každopádně 12 bodů):
  * (až +2 body) U titulů se pokuste rozlišit i různé stupně vzdělání: bakalářský, magisterský, doktorský a vyšší, vojenská hodnost atp. Zkuste odhadnout i podíl žen na kandidátkách.
  * (až +4 body) Pokuste se u jednotlivých kandidátů zjistit, zda kandidovali ve více volbách. Najděte 10 nejpilnějších kandidátů a vypište jejich volební zisky a za jaké strany kandidovali.
  * (až +2 body) Najděte nějaký balíček, který Vám dovolí do Vašeho notebooku zavést interaktivní prvky, např. si vyberete v select-boxu stranu a Váš notebook zobrazí grafy pouze pro ni atp.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte nejen Jupyter Notebook, ale i _csv_ soubor(y) se staženými daty.
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni.

In [84]:
### odtud už je to Vaše
import numpy as np
import pandas as pd
import sklearn as skit
import matplotlib.pyplot as plt
import seaborn as sns


years = ['2006', '2010', '2014', '2018'] #pro pohodlnost parsingu a prace s jednotlive roky udelam pole vsech roku
yearsa = ['2002','2006', '2010', '2014', '2018']
url = 'https://www.volby.cz/pls/kv2002/kv1211?xjazyk=CZ&xid=0&xv=1&xdz=3&xnumnuts=3203'
musor = pd.read_html(url,flavor='html5lib') #udelame list neosetrenych dat pro parsing
musor[1] = musor[1].rename(columns={"Volební strana": "Jednotlive strany","v %":"Pocet hlasu v %", "abs.": "Pocet hlasu lidi"})
procs = [musor[1]['Hlasy'].iloc[:,1]] #taky zakladam list pro chovani poctu hlasu v procentach, protoze
#od 2006 do 2018 roku je tam nespravny format(int64 za misto float), tam ho prepracuju
i=0
v=4
k=1
while i != 4:
    url = 'https://www.volby.cz/pls/kv'
    url += years[i] #jednoduchy parsing
    url += '/kv1111?xjazyk=CZ&xid=0&xdz=3&xnumnuts=3203&xobec=554791&xstat=0&xvyber=0'
    musor += pd.read_html(url,flavor='html5lib')
    if v== 4 or v == 6 or v == 8 or v == 10:
        musor[v] = musor[v].rename(columns={"název": "Jednotlive strany", "v %":"Pocet hlasu v %", "abs.": "Pocet hlasu lidi"}) 
        nnpc = musor[v]['Hlasy'].iloc[:,1] #prepracovani sloupce poctu v %
        npc = nnpc.astype(float)
        npc*=0.01
        procs.append(npc) #ulozeni
    i += 1
    k+=1
    v+=2
i=4
main = [musor[1]['Jednotlive strany'],musor[4]['Kandidátní listina'].iloc[:,[1]],musor[6]['Kandidátní listina'].iloc[:,[1]],musor[8]['Kandidátní listina'].iloc[:,[1]],musor[10]['Kandidátní listina'].iloc[:,[1]]]
lide = [musor[1]['Hlasy'].iloc[:,0], musor[4]['Hlasy'].iloc[:,0],musor[6]['Hlasy'].iloc[:,0],musor[8]['Hlasy'].iloc[:,0],musor[10]['Hlasy'].iloc[:,0]]
al=pd.concat(main, keys = yearsa)
al.index.name = 'Roky voleb'
ld=pd.concat(lide, keys = yearsa)
ld.index.name = 'Roky voleb'
pc=pd.concat(procs, keys = yearsa) 
pc.index.name = 'Roky voleb'
al.insert(1,'Pocet hlasu lidi', ld ) #spojime vsechno dohromady
al.insert(2, 'Pocet hlasu v %', pc)
al.index.names = ['Rok', 'Cislo']
al.reset_index(level=1, inplace=True)
del al['Cislo']
#display(al) #mame krasnou tabulku c.1 
al.to_csv(r'jednstr.csv')
display(pd.read_csv('jednstr.csv'))
#udelal jsem to tak 'trochu' blbe, ale ted' uz vim ze existuje prikaz df.columns.droplevel()


,Rok,Jednotlive strany,Pocet hlasu lidi,Pocet hlasu v %
0,2002,Křesťanská a demokratická unie - Československ...,145 680,7.43
1,2002,Strana zelených,71 830,3.66
2,2002,Česká strana sociálně demokratická,403 889,20.59
3,2002,Občanská demokratická aliance,26 005,1.33
4,2002,Strana za životní jistoty,8 820,0.45
5,2002,Komunistická strana Čech a Moravy,260 123,13.26
6,2002,Občanská demokratická strana,743 089,37.89
7,2002,Pravá volba pro Plzeň,183 245,9.34
8,2002,"Koalice US-DEU, SNK",118 713,6.05
9,2006,Plzeňská aliance,168 750,7.58


In [85]:
url = 'https://www.volby.cz/pls/kv2002/kv21111?xjazyk=CZ&xid=0&xv=11&xdz=3&xnumnuts=3203&xobec=554791&xstrana=0'
jnk = pd.read_html(url,flavor='html5lib')
del jnk[0]['Kandidátnílistina']
i = 0
while i != 4:
    url = 'https://www.volby.cz/pls/kv'
    url += years[i]
    url += '/kv21111?xjazyk=CZ&xid=0&xv=11&xdz=3&xnumnuts=3203&xobec=554791&xstrana=0'
    jnk += pd.read_html(url,flavor='html5lib')
    del jnk[i+1]['Kandidátní listina']
    i+=1
#display(jnk[0].iloc[:,[1,2,3,4,5,6,7,0,8,9]])
k = jnk[0].iloc[:,[1,2,3,4,5,6,7,0,8,9]]
k.columns = k.columns.droplevel()
k = k.replace({pd.np.nan: ' '})

In [86]:
import re
lol = [k, jnk[1].iloc[:,[1,2,3,4,5,6,7,0,8,]], jnk[2].iloc[:,[1,2,3,4,5,6,7,0,8,]], jnk[3].iloc[:,[1,2,3,4,5,6,7,0,8,]],jnk[4].iloc[:,[1,2,3,4,5,6,7,0,8,]]]
z = 0
while z != 5 :
    if z == 0:
        lol[z] = lol[z].rename(columns = {'abs.' : "Pocet hlasu lidi", "Tituly":'Titul', 'v %': "Pocet hlasu v %", "Poř.číslo": "Pořadí kand.", "Pořadí": "Pořadí zvoleni"})
    else:
        lol[z].columns = lol[z].columns.droplevel()
        n = lol[z]['příjmení, jméno, tituly'].str.split(' ').str[:2]
        n = n.apply(' '.join)
        kc = lol[z]['příjmení, jméno, tituly'].str.split(' ').str[2]
        kc = kc.replace({pd.np.nan: ' '})
        kc = kc.astype(str)
        #display(kc)
        lol[z].iloc[:,[0]] = n
        lol[z].insert(1, 'Titul', kc)
        lol[z] = lol[z].rename(columns={"příjmení, jméno, tituly": "Příjmení, jméno","Politickápříslušnost" : "Polit.přísl.", "věk": "Věk", "Navrhujícístrana" :"Navrh.strana" ,"Politickápříslušnost":"Polit.přísl.","abs." :"Pocet hlasu lidi", "v %": "Pocet hlasu v %", "Pořadízvolení/náhradníka" :"Pořadí kand.", "poř.číslo" : "Pořadí zvoleni"})
        kk = lol[z]['Pocet hlasu v %'].astype(float)
        kk*=0.01
        lol[z] = lol[z].drop(columns = ['Pocet hlasu v %'])
        lol[z].insert(6, 'Pocet hlasu v %', kk)
    z+=1
y = pd.concat(lol, keys = yearsa)
y.to_csv(r'kandstmp.csv')
final=pd.read_csv('kandstmp.csv')
del final['Unnamed: 1']
final = final.rename(columns = {'Unnamed: 0': "Rok"})
final.to_csv(r'kands.csv', index = 0)
display(pd.read_csv('kands.csv'))

,Rok,"Příjmení, jméno",Titul,Věk,Navrh.strana,Polit.přísl.,Pocet hlasu lidi,Pocet hlasu v %,Pořadí kand.,Pořadí zvoleni,Mandát
0,2002,Adamec Pavel,MUDr.,35,US-DEU,US-DEU,2 474,2.08,29,26.0,
1,2002,Aloy Zdeněk,Ing.,70,ČSSD,ČSSD,7 283,1.80,45,35.0,
2,2002,Anderle Jan,Ing.,46,PVP,PVP,5 358,2.92,3,3.0,*
3,2002,Andrlík Roman,,35,ČSSD,ČSSD,7 665,1.89,43,33.0,
4,2002,Anton Jiří,,27,SZ,BEZPP,1 343,1.86,16,,
5,2002,Arnošt Patrik,,30,US-DEU,US-DEU,2 476,2.08,13,10.0,
6,2002,Aschenbrenner Lumír,Ing.,42,ODS,ODS,17 550,2.36,17,3.0,*
7,2002,Aust Jan,,71,KSČM,KSČM,5 068,1.94,47,39.0,
8,2002,Babnič Jan,,48,ČSSD,ČSSD,8 220,2.03,34,24.0,
9,2002,Ballýová Hana,,61,KSČM,KSČM,5 405,2.07,40,32.0,
